# 2 - Cleanup
Now that I have merged the downloaded data, I can start studying it to solve the issues thtat can by solved by deleting things.

In [11]:
import pandas as pd


# data = pd.read_csv('data\set_movies_sample_big.csv')
data = pd.read_csv('data\imdb_full - 34k.csv')

In [12]:
data

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,index.1,rating,genres.1,mpaa,description,votes
0,23130,tt0023526,movie,Stranger in Town,Stranger in Town,0,1932,\N,65,Drama,0,6.1,['drama'],TV-PG,"[""Ulysses Crickle owns a small town grocery st...",115.0
1,23131,tt0023527,movie,Strangers in Love,Strangers in Love,0,1932,\N,76,"Comedy,Drama",1,6.7,"['comedy', 'drama']",NaN,['_Strangers in Love_ (qv) is a 1932 American ...,144.0
2,23132,tt0023528,movie,Strangers of the Evening,Strangers of the Evening,0,1932,\N,70,"Comedy,Mystery",2,5.0,"['comedy', 'mystery']",Passed,['Bodies start mysteriously disappearing from ...,218.0
3,23133,tt0023529,movie,Street of Women,Street of Women,0,1932,\N,59,"Drama,Romance",3,6.0,"['drama', 'romance']",Passed,"[""A married architect, stuck in a loveless mar...",356.0
4,23134,tt0023530,movie,Spoiling the Game,Strich durch die Rechnung,0,1932,\N,99,Comedy,4,6.6,['comedy'],NaN,['Whoever of a group of eager bicycle racers f...,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33995,68713,tt0070129,movie,La grande nouba,La grande nouba,0,1974,\N,95,Comedy,995,3.5,['comedy'],NaN,['The viscount Galmiche de Quoibedec and his w...,22.0
33996,68714,tt0070130,movie,La Grande Bouffe,La grande bouffe,0,1973,\N,130,"Comedy,Drama",996,7.1,"['comedy', 'drama']",NC-17,['A group of men go to a villa in the French c...,13058.0
33997,68715,tt0070131,movie,Lifesize,Grandeur nature,0,1974,\N,101,"Comedy,Drama,Fantasy",997,6.4,"['comedy', 'drama', 'fantasy']",X,['Michel is a middle aged man with a troubled ...,377.0
33998,68716,tt0070132,movie,Big Sentiments Make for Good Sports,Les grands sentiments font les bons gueuletons,0,1973,\N,105,Comedy,998,6.2,['comedy'],NaN,NaN,35.0


## Columns
According to the definition in https://www.imdb.com/interfaces/ movies don't have an 'End Year', as that value is reserved for series. As such, this column has no use for me.

In [13]:
# data.endYear.value_counts()


In [14]:
cols_to_drop = []
cols_to_drop.append('endYear')

The column ``titleType`` has only movie as values, because I already filtered anything else at the beggining.

In [15]:
cols_to_drop.append('titleType')

The ``index`` column is here as a small mistake when merging the sources and can be dropped without issue. It happens 3 times, but they all will drop at the same time.
The ``tconst`` column, even though it will serve no purpose at modeling because it is just the ID in IMDb, will be kept in case I need to expand information in the future.

In [16]:
cols_to_drop.append('index')

I have two columns of genders, but the second one has the same information but in a better format, so I will delete the other.

In [17]:
# cols_to_drop.append('genres')


There are also two columns for names and they won't affect much, but still I will keep the names in English from ''primaryTitle''.

In [18]:
cols_to_drop.append('originalTitle')

Now it's time to drop some columns.

In [19]:
data = data.drop(cols_to_drop, 1)
data.head(1)

C:\Users\carlo\AppData\Local\Temp/ipykernel_22640/1662013735.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(cols_to_drop, 1)


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,rating,genres.1,mpaa,description,votes
0,tt0023526,Stranger in Town,0,1932,65,6.1,['drama'],TV-PG,"[""Ulysses Crickle owns a small town grocery st...",115.0


## NaNs

In [20]:
data.isna().sum()/len(data)

tconst            0.000000
primaryTitle      0.000000
isAdult           0.000000
startYear         0.000000
runtimeMinutes    0.000000
rating            0.104029
genres.1          0.018882
mpaa              0.531118
description       0.177235
votes             0.104029
dtype: float64

From this overview I can get some conclussions:  
- The first columns are all populated because the come from an official database  
- the last 2 columns appear as filled with errors due to a problem with the scraper, I will try to solve it later.

### ``rating``
This column is going to be one of my targets, so I need to keep the actual data and remove the rest.

In [21]:
data = data.dropna(subset=['rating'])

In [22]:
data.isna().sum()/len(data)

tconst            0.000000
primaryTitle      0.000000
isAdult           0.000000
startYear         0.000000
runtimeMinutes    0.000000
rating            0.000000
genres.1          0.000000
mpaa              0.485638
description       0.116929
votes             0.000000
dtype: float64

### ``mpaa``
The age rating has a list of defined values: 
* **G – General Audiences**
* **PG – Parental Guidance Suggested**
* **PG-13 – Parents Strongly Cautioned**
* **R – Restricted**
* **NC-17 – Adults Only**


In [23]:
data.mpaa.value_counts(dropna=False)

NaN          14794
Approved      6161
Passed        3655
Not Rated     2588
R              832
PG             445
TV-PG          426
G              357
Unrated        327
GP             224
TV-G           179
X              130
TV-14          102
M               90
M/PG            60
PG-13           39
TV-MA           33
TV-Y7            7
TV-Y7-FV         4
NC-17            4
Open             2
TV-Y             2
TV-13            2
Name: mpaa, dtype: int64


For now, I will assign the empty values to the most common appearance, that is Not Rated.
The rest of the changes will come later.

In [24]:
data.mpaa.fillna('Not Rated', inplace=True)
# data.mpaa.value_counts(dropna=False)

### ``description``
As the most important column, I have no use for rows with no information in it, so I need to remove them.

In [25]:
data = data.dropna(subset=['description'])

Lastly, the columns that come from Rotten Tomatoes have to be deleted, as I have lost access to the web.

In [26]:
# data = data.drop(['rt_critics_score', 'rt_audience_score'], 1)
# data.head(1)

C:\Users\carlo\AppData\Local\Temp/ipykernel_22640/3169075514.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(['rt_critics_score', 'rt_audience_score'], 1)


KeyError: "['rt_critics_score' 'rt_audience_score'] not found in axis"

In [27]:
# data.to_csv('data\movies_clean.csv', index=False)
data.to_csv('data\sample_clean.csv', index=False)